In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup

from tqdm.auto import tqdm

In [ ]:
TRAIN_DATA_PATH = "../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv"
VALID_DATA_PATH = "../input/jigsaw-multilingual-toxic-comment-classification/validation.csv"
TEST_DATA_PATH = "../input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv"

In [ ]:
def read_data(dirs):
    data = (pd.read_csv(dirs[i]) for i in range(len(dirs)))
    return data

df_train,df_valid,df_test = read_data([TRAIN_DATA_PATH,VALID_DATA_PATH,TEST_DATA_PATH])

In [ ]:
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].mean(axis=1)

df_train['y'] = df_train['score']

min_len = (df_train['y'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments
df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df
df_train_new

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

raw_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
raw_tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_train_new[['comment_text']])

def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["comment_text"]

In [ ]:
raw_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

## Training

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression

In [ ]:
def dummy_fun(doc):return doc

labels = df_train_new['y']
comments = df_train_new['comment_text']
tokenized_comments = tokenizer(comments.to_list())['input_ids']

vectorizer = TfidfVectorizer(
    analyzer = 'word',
    tokenizer = dummy_fun,
    preprocessor = dummy_fun,
    token_pattern = None)

comments_tr = vectorizer.fit_transform(tokenized_comments)
# print(comments_tr)

In [ ]:
regressor = LinearRegression()
regressor.fit(comments_tr, labels)

## Validation

In [ ]:
# preprocess val data
less_toxic_comments = df_valid[df_valid['toxic'] == 0]['comment_text']
more_toxic_comments = df_valid[df_valid['toxic'] == 1]['comment_text']

less_toxic_comments = tokenizer(less_toxic_comments.to_list())['input_ids']
more_toxic_comments = tokenizer(more_toxic_comments.to_list())['input_ids']

less_toxic = vectorizer.transform(less_toxic_comments)
more_toxic = vectorizer.transform(more_toxic_comments)

# make predictions
y_pred_less = regressor.predict(less_toxic)
y_pred_more = regressor.predict(more_toxic)
print(y_pred_more)

In [ ]:
texts = df_test['comment_text']
texts = tokenizer(texts.to_list())['input_ids']
texts = vectorizer.transform(texts)

In [ ]:
df_test['prediction'] = regressor.predict(texts)
df_test = df_test[['comment_text','prediction']]

df_test['score'] = df_test['prediction']
df_test = df_test[['comment_text','score']]

In [ ]:
df_test.head(10)